In [7]:
%pip install pandas

import pandas as pd

booking_part_1 = pd.read_csv('data/booking/bookingData-2024-06.csv')
booking_part_2 = pd.read_csv('data/booking/bookingData-2024-09.csv')
booking_part_3 = pd.read_csv('data/booking/bookingData-2024-12.csv')
booking_part_4 = pd.read_csv('data/booking/bookingData-2024-12-2.csv')
booking_part_5 = pd.read_csv('data/booking/bookingData-2024-1.csv')
booking_part_6 = pd.read_csv('data/booking/bookingDataServer.csv')

booking = pd.concat([booking_part_1, booking_part_2, booking_part_3, booking_part_4, booking_part_5, booking_part_6])
booking['price'] = booking['price'].str.replace('€', '').str.replace(' ', '').str.replace(',','.').astype(float)

booking_data = booking[['month', 'year', 'lat', 'lng', 'price', 'room type']].rename(columns={'lat': 'latitude','lng': 'longitude', 'room type': 'room_type'})

booking = booking.drop_duplicates()

booking_data.to_csv('data/booking/bookingData.csv', index=False)

Note: you may need to restart the kernel to use updated packages.


In [8]:
airbnb = pd.read_csv('airbnb_data.csv')

airbnb["check_in"] = pd.to_datetime(airbnb["check_in"])
airbnb["check_out"] = pd.to_datetime(airbnb["check_out"])

airbnb['month'] = airbnb['check_in'].dt.month
airbnb['year'] = airbnb['check_in'].dt.year

airbnb_data = airbnb[['month', 'year', 'latitude', 'longitude', 'price', 'room_type']]

In [9]:
data = pd.concat([booking_data, airbnb_data])
data.to_csv('data.csv', index=False)

In [10]:
data = pd.read_csv('data.csv')
data_graph = data.groupby('month')['price'].mean().reset_index()
data_graph['price'] = data_graph['price'].round(2)
data_graph.to_csv('data_graph.csv', index=False)

In [11]:
%pip install geopy
from geopy.geocoders import Nominatim

def get_district(latitude, longitude):
    # Initialize Nominatim API
    geolocator = Nominatim(user_agent="map_app_airbnb", timeout=720000)

    # Get location with reverse geocode
    location = geolocator.reverse((latitude, longitude), exactly_one=True)

    if location:
        address = location.raw['address']
        district = address.get('city_district')
        if district == None:
            district = address.get('suburb')         
        return district
    else:
        return "District not found"

# Example usage
latitude = 40.748817
longitude = -73.985428
print(get_district(40.3789571457818,-3.6703623401582193))

Note: you may need to restart the kernel to use updated packages.
Puente de Vallecas


In [12]:
data_map = data.groupby(['latitude', 'longitude', 'month'])['price'].mean().reset_index()
data_map['region'] = data_map.apply(lambda x: get_district(x['latitude'], x['longitude']), axis=1)

data_map.to_csv('data_map.csv', index=False)

In [13]:
data_map.groupby(['region', 'month'])['price'].mean().reset_index().to_csv('data_map_agg.csv', index=False)